In [2]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
pd.set_option("max_rows", 500)
pd.set_option("max_columns", 500)
pd.set_option('float_format', '{:f}'.format)

# import dataload

import os 
import random
import datetime
from tqdm import tqdm 
from functools import reduce

from sklearn.preprocessing import StandardScaler

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt 

plt.style.use('seaborn')
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

In [3]:
cd /content/drive/MyDrive/2021/날씨 공모전/code

/content/drive/MyDrive/2021/날씨 공모전/code


In [7]:
food = pd.read_csv('../data/food_tsclustering_norm_0613.csv', encoding='cp949')
product = pd.read_csv('../data/final_weather_0613.csv', encoding='cp949')
weather = pd.read_csv("../data/weather_final2.csv", encoding="cp949")

food.shape, product.shape, weather.shape

((94, 367), (1303336, 52), (11680, 29))

In [8]:
tqdm.pandas()
product['date'] = product['date'].progress_apply(lambda x : pd.to_datetime(str(x), format='%Y-%m-%d'))
weather['날짜'] = weather['날짜'].progress_apply(lambda x : pd.to_datetime(str(x), format='%Y-%m-%d'))
weather = weather.rename(columns={"날짜" : "date"})

100%|██████████| 11680/11680 [00:01<00:00, 11625.46it/s]


In [9]:
def region_weather(loc):
    weather_true = weather.drop(['연', '월', '일', '분기', '요일', '공휴일명', '공휴일여부', '주말여부', '계절', 'year', 'month'], axis=1) 
    weather1 = pd.pivot_table(weather_true[weather_true['지역'] == loc], index='date', columns='지역').reset_index(level=0)
    weather1.columns = [l+"_"+w for l,w in zip(weather1.columns.get_level_values(1), weather1.columns.get_level_values(0))]
    return weather1.rename(columns = {'_date' : 'date'})

In [21]:
def get_dummies(data, var_list):
    # var_list = ['var1', 'var2']
    for var in var_list:
        if var=='sex':
            data[var] = pd.get_dummies(data[var]).iloc[:,:1] # F(1), M(0)
        else:
            data = pd.concat([data.drop(var, axis=1), pd.get_dummies(data[var], prefix=var, drop_first=True)], axis=1)
    return data

In [37]:
def cluster_df(big_cat, product_cat) : 
    
    '''
    big_cat : TimeSeries Clustering 결과 파일 (pivot table 형태의 dataframe)
    product_cat : big_category 에 해당되는 dataframe 
    '''
    
    cluster0 = big_cat[big_cat["cluster"] == 0] 
    cluster1 = big_cat[big_cat["cluster"] == 1]
    cluster2 = big_cat[big_cat["cluster"] == 2]
    
    cluster0 = list(cluster0['sm_cat']) 
    cluster1 = list(cluster1['sm_cat']) 
    cluster2 = list(cluster2['sm_cat']) 
    
    big_c0 = product_cat[product_cat['sm_cat'].isin(cluster0)].reset_index(drop=True).drop('big_cat', axis=1)
    big_c1 = product_cat[product_cat['sm_cat'].isin(cluster1)].reset_index(drop=True).drop('big_cat', axis=1)
    big_c2 = product_cat[product_cat['sm_cat'].isin(cluster2)].reset_index(drop=True).drop('big_cat', axis=1)

    big_c0 = get_dummies(big_c0, ['sm_cat'])
    big_c1 = get_dummies(big_c1, ['sm_cat'])
    big_c2 = get_dummies(big_c2, ['sm_cat'])

    big_c0 = pd.concat([big_c0[['date', 'qty']], big_c0.drop(['date', 'qty'], axis=1)], axis=1)
    big_c1 = pd.concat([big_c1[['date', 'qty']], big_c1.drop(['date', 'qty'], axis=1)], axis=1)
    big_c2 = pd.concat([big_c2[['date', 'qty']], big_c2.drop(['date', 'qty'], axis=1)], axis=1)
    
    return big_c0, big_c1, big_c2

In [38]:
food_c0, food_c1, food_c2 = cluster_df(food, product)
food_c0.shape, food_c1.shape, food_c2.shape

((164709, 72), (344022, 97), (164912, 72))

In [39]:
food_c0.head()

,date,qty,sex,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,공휴일여부,주말여부,연_2019,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,분기_2,분기_3,분기_4,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6,계절_1,계절_2,계절_3,age_20,age_30,age_40,age_50,sm_cat_견과류 땅콩,sm_cat_기타 농산물,sm_cat_나물,sm_cat_녹차,sm_cat_느타리버섯,sm_cat_닭 양념육,sm_cat_둥굴레차,sm_cat_딸기우유,sm_cat_루테인/눈 영양제,sm_cat_배/포도/과일즙,sm_cat_보리차,sm_cat_부추,sm_cat_야채/호박즙,sm_cat_양배추/양상추,sm_cat_오메가3/스쿠알렌 영양제,sm_cat_유제품 음료,sm_cat_인스턴트커피,sm_cat_전통차,sm_cat_캡슐/POD커피,sm_cat_프로폴리스/로얄젤리,sm_cat_해조류 다시마,sm_cat_홍차
0,2018-01-01,6.000000,1,32.355555,10.693870,4.091370,0.000000,0.000000,0.044400,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018-01-01,17.000000,1,23.352080,12.429690,6.352940,0.000000,0.512820,0.424170,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018-01-01,18.000000,1,10.889325,13.022490,4.880370,0.000000,0.406500,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2018-01-01,6.000000,1,9.713200,11.690800,4.533500,1.219510,0.909090,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2018-01-01,0.000000,1,3.448270,13.927120,5.478870,0.000000,0.000000,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
# 지역별로 붙여서 모델링 
def loc_df(cluster) : 
    
    '''
    - 수도권: 서울
    - 강원: 강릉
    - 경북: 대구
    - 경남: 부산
    - 전남: 광주
    - 전북: 전주 
    - 충남: 대전
    - 충북: 청주 
    - 제주: 제주
    '''
    
    c0_1 = cluster.merge(region_weather('서울'), on="date", how='left')
    c0_2 = cluster.merge(region_weather('강릉'), on="date", how='left')
    c0_3 = cluster.merge(region_weather('대구'), on="date", how='left')
    c0_4 = cluster.merge(region_weather('부산'), on="date", how='left')
    c0_5 = cluster.merge(region_weather('광주'), on="date", how='left')
    c0_6 = cluster.merge(region_weather('전주'), on="date", how='left')
    c0_7 = cluster.merge(region_weather('대전'), on="date", how='left')
    c0_8 = cluster.merge(region_weather('청주'), on="date", how='left')
    c0_9 = cluster.merge(region_weather('제주'), on="date", how='left')  
    
    return c0_1, c0_2, c0_3, c0_4, c0_5, c0_6, c0_7, c0_8, c0_9

In [40]:
food_c0_1, food_c0_2, food_c0_3, food_c0_4, food_c0_5, food_c0_6, food_c0_7, food_c0_8, food_c0_9 = loc_df(food_c0) 
food_c0_1.shape, food_c0_2.shape, food_c0_3.shape, food_c0_4.shape, food_c0_5.shape, food_c0_6.shape, food_c0_7.shape, food_c0_8.shape, food_c0_9.shape

((164709, 88),
 (164709, 88),
 (164709, 88),
 (164709, 88),
 (164709, 88),
 (164709, 88),
 (164709, 88),
 (164709, 88),
 (164709, 88))

In [41]:
food_c0_1.head()

,date,qty,sex,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,공휴일여부,주말여부,연_2019,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,분기_2,분기_3,분기_4,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6,계절_1,계절_2,계절_3,age_20,age_30,age_40,age_50,sm_cat_견과류 땅콩,sm_cat_기타 농산물,sm_cat_나물,sm_cat_녹차,sm_cat_느타리버섯,sm_cat_닭 양념육,sm_cat_둥굴레차,sm_cat_딸기우유,sm_cat_루테인/눈 영양제,sm_cat_배/포도/과일즙,sm_cat_보리차,sm_cat_부추,sm_cat_야채/호박즙,sm_cat_양배추/양상추,sm_cat_오메가3/스쿠알렌 영양제,sm_cat_유제품 음료,sm_cat_인스턴트커피,sm_cat_전통차,sm_cat_캡슐/POD커피,sm_cat_프로폴리스/로얄젤리,sm_cat_해조류 다시마,sm_cat_홍차,서울_1시간최대강수량,서울_1시간최대습도,서울_PM10,서울_PM25,서울_강수여부,서울_열지수,서울_일별강수량,서울_일조시간합,서울_지점번호,서울_체감온도,서울_최고기온,서울_최고현지기압,서울_최저기온,서울_평균기온,서울_평균풍속,서울_폭염여부
0,2018-01-01,6.000000,1,32.355555,10.693870,4.091370,0.000000,0.000000,0.044400,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,57.100000,42.307692,21.470696,0,-44.993999,0.000000,51.600000,108,9.649709,3.800000,1018.100000,-5.100000,-1.300000,1.400000,0
1,2018-01-01,17.000000,1,23.352080,12.429690,6.352940,0.000000,0.512820,0.424170,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,57.100000,42.307692,21.470696,0,-44.993999,0.000000,51.600000,108,9.649709,3.800000,1018.100000,-5.100000,-1.300000,1.400000,0
2,2018-01-01,18.000000,1,10.889325,13.022490,4.880370,0.000000,0.406500,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,57.100000,42.307692,21.470696,0,-44.993999,0.000000,51.600000,108,9.649709,3.800000,1018.100000,-5.100000,-1.300000,1.400000,0
3,2018-01-01,6.000000,1,9.713200,11.690800,4.533500,1.219510,0.909090,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,57.100000,42.307692,21.470696,0,-44.993999,0.000000,51.600000,108,9.649709,3.800000,1018.100000,-5.100000,-1.300000,1.400000,0
4,2018-01-01,0.000000,1,3.448270,13.927120,5.478870,0.000000,0.000000,0.000000,0.885198,5.464146,-2.880740,0.000000,0.000000,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,57.100000,42.307692,21.470696,0,-44.993999,0.000000,51.600000,108,9.649709,3.800000,1018.100000,-5.100000,-1.300000,1.400000,0
